In [1]:
import dt_apriltags as atag
import cv2
import numpy as np


# test images camera parameters
fx, fy, cx, cy = (867.1714530943214, 7.2700778250832485, 518.2694115721059, 910.7793844438535)
camera_params = [fx,fy,cx,cy]


In [2]:
def combine_rt(r,t):
    return np.array([
        [r[0,0],r[0,1],r[0,2],t[0,0]],
        [r[1,0],r[1,1],r[1,2],t[1,0]],
        [r[2,0],r[2,1],r[2,2],t[2,0]],
        [0,0,0,1]
    ])

def scale_vec(origin,point,scale):
    diffs = point-origin
    length = np.linalg.norm(diffs)
    return origin - (diffs/length*scale)

def draw_pose(overlay, camera_params, tag_size, pose_r, pose_t, linescale = 2, linewidth = 5, z_sign=1, label = None):
# def draw_pose(overlay, camera_params, tag_size, pose, width = 2, linescale = 2, z_sign=1):
    opoints = np.array([
        0, 0, 0,
        1, 0, 0,
        0, 1, 0,
        0, 0, -1 * z_sign,
        0, -2/linescale, 0,
    ]).reshape(-1, 1, 3) * 0.5 * tag_size * linescale
    boxpoints = np.array([
        -1,-1,0,
        -1,1,0,
        1,1,0,
        1,-1,0
    ]).reshape(-1, 1, 3) * 0.5 * tag_size

    fx, fy, cx, cy = camera_params

    K = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1]
        ])

    # rvec, _ = cv2.Rodrigues(pose_r)
    # tvec = pose_t

    # rvec, _ = cv2.Rodrigues(pose[:3, :3])
    # tvec = pose[:3, 3]

    dcoeffs = np.zeros(5)

    ipoints, _ = cv2.projectPoints(opoints, pose_r, pose_t, K, dcoeffs)

    
    ipoints = np.round(ipoints).astype(int)
    ipoints = [tuple(pt) for pt in ipoints.reshape(-1, 2)]

    textsize = abs(ipoints[0][0]-ipoints[1][0])

    # print(boxpoints)

    cv2.arrowedLine(overlay, ipoints[0], ipoints[1], (0,0,255), linewidth)
    cv2.arrowedLine(overlay, ipoints[0], ipoints[2], (0,255,0), linewidth)
    # cv2.arrowedLine(overlay, ipoints[0], ipoints[3], (255,0,0), linewdith)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(overlay, ' X', ipoints[1], font, textsize/50, (0,0,255), round(linewidth/2),cv2.LINE_AA)
    cv2.putText(overlay, ' Y', ipoints[2], font, textsize/50, (0,255,0), round(linewidth/2),cv2.LINE_AA)
    if label:
        textwidth = cv2.getTextSize(label,font,textsize/25, round(linewidth/2))[0][0]/2
        ori = (round(ipoints[4][0]-textwidth),ipoints[4][1])
        cv2.putText(overlay, label, ori,font, textsize/25, (50,255,50), round(linewidth/2),cv2.LINE_AA)


    # _draw_cube(overlay,camera_params,tag_size,pose_r,pose_t,z_sign)

def _draw_cube(overlay, camera_params, tag_size, pose_r,pose_t, z_sign=1):

    opoints = np.array([
        -1,-1,0,
        1,-1,0,
        1,1,0,
        -1,1,0,
        # -1,-1,2 * z_sign,
        # 1,-1,2 * z_sign,
        # 1,1,2 * z_sign,
        # -1,1, 2 * z_sign
    ]).reshape(-1, 1, 3) * 0.5 * tag_size

    edges = np.array([
        0, 1,
        1, 2,
        2, 3,
        3, 0,
        # 0, 4,
        # 1, 5,
        # 2, 6,
        # 3, 7,
        # 4, 5,
        # 5, 6,
        # 6, 7,
        # 7, 4
    ]).reshape(-1, 2)

    fx, fy, cx, cy = camera_params

    K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
    ])

    # rvec, _ = cv2.Rodrigues(pose[:3, :3])
    # tvec = pose[:3, 3]

    dcoeffs = np.zeros(5)

    ipoints, _ = cv2.projectPoints(opoints, pose_r, pose_t, K, dcoeffs)

    ipoints = np.round(ipoints).astype(int)

    ipoints = [tuple(pt) for pt in ipoints.reshape(-1, 2)]

    for i, j in edges:
        cv2.line(overlay, ipoints[i], ipoints[j], (0, 255, 0), 1, 16)


def get_tag_pixel_size(detection_object):
    corners = np.array(detection_object.corners)
    xdiffs = [
        abs(corners[0,0]-corners[1,0]),
        abs(corners[0,0]-corners[2,0]),
        abs(corners[3,0]-corners[1,0]),
        abs(corners[3,0]-corners[2,0]),
    ]
    ydiffs = [
        abs(corners[0,1]-corners[1,1]),
        abs(corners[0,1]-corners[2,1]),
        abs(corners[3,1]-corners[1,1]),
        abs(corners[3,1]-corners[2,1]),
    ]
    return max(max(xdiffs),max(ydiffs))

Coordinate System

The coordinate system has the origin at the camera center. The z-axis points from the camera center out the camera lens. The x-axis is to the right in the image taken by the camera, and y is down. The tag's coordinate frame is centered at the center of the tag, with x-axis to the right, y-axis down, and z-axis into the tag.

In [3]:
# read in image
# image = cv2.imread("testimages/cal1.jpg")
# image = cv2.imread("testimages/tag3blurred.jpg")
# image = cv2.imread("testimages/tag6a.jpg")
# image = cv2.imread("realsense/calnear_Color.png")
image = cv2.imread("testimages/view300_300.jpg")

# image = cv2.blur(image,(10,10))

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# create detector
detector = atag.Detector(families="tagStandard41h12",
                        #  quad_sigma=1,
                        # max_hamming=5,
                        # refine_edges=10

)

size = 0.2032
# size = 0.0127

linewidth = 15
linescale = 2

# detect apriltag
results = detector.detect(gray,True,camera_params,size)

for tag in results:
    # print(tag)

    p_size = get_tag_pixel_size(tag)


    corners = np.array(tag.corners, np.int32)
    corners = corners.reshape((-1,1,2))
    cv2.polylines(image,[corners],True,(255,0,0),linewidth)

    o = np.atleast_2d([[1],[1],[1]])
    r = np.array(tag.pose_R)
    t = np.array(tag.pose_t)

    rvec, _ = cv2.Rodrigues(r)

    print("tag: ",tag.tag_id)
    # print("o",o)
    print("r",r)
    print("rvec",rvec)
    print("t",t)
    # print(r @ t )
    rt = combine_rt(r,t)
    # print(rt)
    # draw_pose(image, camera_params,0.2032,rt,5,1,1)
    draw_pose(image, camera_params,tag_size= size,pose_r=r,pose_t=t,linescale=linescale,linewidth=linewidth,z_sign=-1, label=str(tag.tag_id))
cv2.imwrite("output.jpg",image)

print(f"Found {len(results)} tags. Results in output.jpg")




tag:  3
r [[ 0.58410934 -0.4698819   0.66183629]
 [-0.81166417 -0.34234655  0.4732865 ]
 [ 0.00418862 -0.81363987 -0.58135412]]
rvec [[-1.99719601]
 [ 1.02061108]
 [-0.53041589]]
t [[ 0.28305201]
 [41.56870387]
 [ 0.48037737]]
tag:  7
r [[ 0.43372468 -0.78981804  0.4336708 ]
 [-0.90104063 -0.37860931  0.21161467]
 [-0.00294528 -0.48253752 -0.87587035]]
rvec [[-2.27732525]
 [ 1.43241907]
 [-0.3648912 ]]
t [[  3.12288914]
 [213.84861458]
 [  1.93303813]]
tag:  9
r [[ 0.52555793 -0.69772841  0.48678941]
 [-0.85073523 -0.43518171  0.29473115]
 [ 0.00619955 -0.5690272  -0.82229533]]
rvec [[-2.26068474]
 [ 1.25783116]
 [-0.40045944]]
t [[  2.78574146]
 [254.43451166]
 [  2.28459668]]
Found 3 tags. Results in output.jpg
